In [16]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from qlora import *
from collections import defaultdict
import copy
import json
from os.path import exists, join, isdir
from dataclasses import dataclass, field
import sys
from typing import Optional, Dict, Sequence
import numpy as np
from tqdm import tqdm
import logging
import bitsandbytes as bnb
import pandas as pd
import importlib
from packaging import version
from packaging.version import parse
import warnings
from sklearn.metrics.pairwise import manhattan_distances
from torchmetrics.functional.pairwise import pairwise_manhattan_distance as manhattan
from torchmetrics.functional.pairwise import pairwise_cosine_similarity as cossim
import numpy as np

import torch
import transformers
from torch.nn.utils.rnn import pad_sequence
import argparse
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    set_seed,
    Seq2SeqTrainer,
    BitsAndBytesConfig,
    LlamaTokenizer

)
from datasets import load_dataset, Dataset, load_from_disk
import evaluate

from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel
)
from peft.tuners.lora import LoraLayer
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

from transformers.modeling_utils import unwrap_model
from transformers.models.auto.modeling_auto import MODEL_FOR_CAUSAL_LM_MAPPING_NAMES
from transformers.utils import is_peft_available
from peft import PeftModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
argdict = {
  'model_name_or_path' : './mhllama', #'./mhllama',
  'num_heads' : 4,
  'data_seed' : 42 ,
  'do_eval': True,
  'eval_dataset_size' : 5 ,
  'max_eval_samples' : 100 ,
  'per_device_eval_batch_size' : 2 ,
  'max_new_tokens' : 60 ,
  'dataloader_num_workers' : 1 ,
  'group_by_length' : True,
  'remove_unused_columns' : False ,
  'lora_r' : 64 ,
  'lora_alpha' : 16 ,
  'lora_modules' : 'all' ,
  'double_quant' : True,
  'quant_type' : 'nf4' ,
  'bf16' : True,
  'bits' : 4 ,
  'dataset' : '/mnt/data/sonia/honeygan/apr23.dat',
  'source_max_len' : 60 ,
  'target_max_len' : 60 ,
  'seed' : 0
}

arglist = [f'--{k}={v}' for k,v in argdict.items()]

In [30]:
hfparser = transformers.HfArgumentParser((
    ModelArguments, DataArguments, TrainingArguments, GenerationArguments
))
model_args, data_args, training_args, generation_args  = hfparser.parse_args_into_dataclasses(args=arglist, return_remaining_strings=True)[:-1]
training_args.generation_config = transformers.GenerationConfig(**vars(generation_args))
args = argparse.Namespace(
    **vars(model_args), **vars(data_args), **vars(training_args)
)

In [19]:
checkpoint_dir, completed_training = get_last_checkpoint(args.output_dir)
model, tokenizer = get_accelerate_model(args, checkpoint_dir)
model.config.use_cache = False
    
print('loaded model')
set_seed(args.seed)

loading base model ./mhllama...


/home/sonia/transformers-4.39.3/src/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of MultiheadLlamaForCausalLM were not initialized from the model checkpoint at ./mhllama and are newly initialized: ['heads.1.weight', 'heads.2.weight', 'heads.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MH llama has 4 heads
Adding special tokens.
adding LoRA modules...
['q_proj', 'heads.2', 'up_proj', 'gate_proj', 'heads.3', 'v_proj', 'down_proj', 'heads.0', 'k_proj', 'o_proj', 'heads.1']
loaded model


In [31]:
data_module = make_data_module(tokenizer=tokenizer, args=args)
collator = data_module['data_collator']
datatr = data_module['train_dataset']

['a ', '<0x00> at IP ', '<0x00>, port ', '<0x00>, offering service ', '<0x00>']
[tensor([263]), tensor([   3,  472, 5641]), tensor([    3, 29892,  2011]), tensor([    3, 29892, 27032,  2669]), tensor([3])]


In [32]:
trainerclass = Seq2SeqTrainer
trainer = trainerclass(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    **{k:v for k,v in data_module.items() if k != 'predict_dataset'},
)
class evalSampleCallback(transformers.TrainerCallback):
    def on_evaluate(self, args, state, control, model, **kwargs):
        trainer.model.eval()
        metrics = trainer.predict(test_dataset=data_module['eval_dataset'],metric_key_prefix="predict")
        
        predictions = []
        for i in range(len(metrics.predictions)):
            logit = metrics.predictions[i]
            label = metrics.label_ids[i] #just to see positions where prompt tokens are at
            logit_abcd = logit[label != IGNORE_INDEX]
            toks = np.argmax(logit_abcd, axis=1)
            predictions.append(
                ''.join(trainer.tokenizer.decode(toks, skip_special_tokens=True, clean_up_tokenization_spaces=True))
                )
        
        for pred in predictions:
            print(pred)
    
    
trainer.add_callback(evalSampleCallback)

/mnt/data/sonia/miniconda3/envs/llama/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [54]:
trainer.evaluate()

targets torch.Size([8, 15])
targets_tok torch.Size([2, 4, 16])
prompt_ids [torch.Size([1]), torch.Size([3]), torch.Size([3]), torch.Size([4]), torch.Size([1])]
input_ids torch.Size([2, 76]) torch.Size([2, 76])
tensor([-1,  1, 20, 39, 59], dtype=torch.int32)
targets torch.Size([8, 14])
targets_tok torch.Size([2, 4, 15])
prompt_ids [torch.Size([1]), torch.Size([3]), torch.Size([3]), torch.Size([4]), torch.Size([1])]
input_ids torch.Size([2, 72]) torch.Size([2, 72])
tensor([-1,  1, 19, 37, 56], dtype=torch.int32)
targets torch.Size([8, 15])
targets_tok torch.Size([2, 4, 16])
prompt_ids [torch.Size([1]), torch.Size([3]), torch.Size([3]), torch.Size([4]), torch.Size([1])]
input_ids torch.Size([2, 76]) torch.Size([2, 76])
tensor([-1,  1, 20, 39, 59], dtype=torch.int32)
targets torch.Size([8, 16])
targets_tok torch.Size([2, 4, 17])
prompt_ids [torch.Size([1]), torch.Size([3]), torch.Size([3]), torch.Size([4]), torch.Size([1])]
input_ids torch.Size([2, 80]) torch.Size([2, 80])
tensor([-1,  1, 

AttributeError: 'NoneType' object has no attribute 'float'

In [ ]:
trainer.model.set_trace(True)

# Trying to load faster (work in progress, crashy)

In [ ]:
if torch.cuda.is_available():
    n_gpus = torch.cuda.device_count()
if is_ipex_available() and torch.xpu.is_available():
    n_gpus = torch.xpu.device_count()
device_map='auto'
max_memory = f'{args.max_memory_MB}MB'
max_memory = {i: max_memory for i in range(n_gpus)}
compute_dtype = (torch.float16 if args.fp16 else (torch.bfloat16 if args.bf16 else torch.float32))

transformers.AutoConfig.register('mhllama', MHLlamaConfig)
transformers.AutoModelForCausalLM.register(MHLlamaConfig, MultiheadLlamaForCausalLM)

In [ ]:
from collections import OrderedDict
sd_llamamodel = transformers.LlamaModel(transformers.LlamaConfig()).state_dict()
headlist = []
for i in range(4):
    headlist.append( torch.nn.Linear(4096, 32000, bias=False) )
heads = torch.nn.ModuleList(headlist)
sd_heads = heads.state_dict()
sd = OrderedDict(list(sd_llamamodel.items()) + list(sd_heads.items()))